# Imports and Constants

In [107]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

from demoparser2 import DemoParser

import os

In [98]:
# Define the path to the 'demos' folder
DEMOS_FOLDER = os.path.join(os.path.dirname(''), "..", "demos")

ASSETS_FOLDER = os.path.join(os.path.dirname(''), "..", "assets")
MAPS_BACKGROUND_FOLDER = os.path.join(ASSETS_FOLDER, "maps_background")

# Load all demos paths
demos_paths = [os.path.join(DEMOS_FOLDER, f) for f in os.listdir(DEMOS_FOLDER) if f.endswith('.dem')]

# Load all map background images paths
maps_background_paths = {f.split('.')[0]: os.path.join(MAPS_BACKGROUND_FOLDER, f) for f in os.listdir(MAPS_BACKGROUND_FOLDER) if f.endswith('.png')}

# Parse Demos

In [99]:
# Parse a demo file
parser = DemoParser(demo_path=demos_paths[0])
header = parser.parse_header()
header['demo_path'] = demos_paths[0]
header['map_background_path'] = maps_background_paths.get(header['map_name'], None)

header

{'demo_version_guid': '8e9d71ab-04a1-4c01-bb61-acfede27c046',
 'client_name': 'SourceTV Demo',
 'allow_clientside_entities': 'true',
 'allow_clientside_particles': 'true',
 'game_directory': '/home/cs2server/cs-gs1/serverfiles/game/csgo',
 'addons': '',
 'fullpackets_version': '2',
 'demo_version_name': 'valve_demo_2',
 'map_name': 'de_train',
 'patch_version': '14126',
 'demo_file_stamp': 'PBDEMS2\x00',
 'server_name': 'StarLadder Major Budapest 2025 | A',
 'demo_path': '../demos/nrg-vs-imperial-m2-train.dem',
 'map_background_path': '../assets/maps_background/de_train.png'}

In [111]:
parser.parse_event('player_death')
weapon_fire_df = parser.parse_event("weapon_fire")

# Get all weapon names in the tick for a each player
ticks_df = parser.parse_ticks(wanted_props=['tick', 'X', 'Y', 'health', 'weapon_name', 'is_freeze_period', 'is_warmup_period', 'team_name', 'round_win_status', 'round_win_reason', 'bomb_planted', 'round_start_time', 'is_bomb_planted', 'game_time'])

In [112]:
# Discard freeze time and warmup time
ticks_df = ticks_df[(ticks_df['is_freeze_period'] == False) & (ticks_df['is_warmup_period'] == False)]

# Drop is_freeze_period and is_warmup_period columns
ticks_df.drop(columns=['is_freeze_period', 'is_warmup_period'], inplace=True)

# Derive seconds elapsed in each round using the parser's round_start_time and game_time fields
ticks_df['seconds_elapsed_in_round'] = (ticks_df['game_time'] - ticks_df['round_start_time']).clip(lower=0)
ticks_df.drop(columns=['round_start_time', 'game_time'], inplace=True)

# Feature Engineering

In [113]:
status_map = {0: "in_progress", 2: "terrorists", 3: "counter_terrorists"}
reason_map = {0: "none", 1: "bomb_exploded", 7: "bomb_defused",
              8: "t_killed", 9: "ct_killed", 12: "time_ran_out"}

ticks_df["round_win_status_label"] = ticks_df["round_win_status"].map(status_map).fillna("unknown")
ticks_df["round_win_reason_label"] = ticks_df["round_win_reason"].map(reason_map).fillna("unknown")

# Keep round_win_status for modeling but drop the numeric reason after mapping
ticks_df.drop(columns=['round_win_reason'], inplace=True)

In [115]:
# Create t_alive and ct_alive columns using transform to align with original index
ticks_df["t_alive"] = ticks_df.groupby("tick")["team_name"].transform(lambda x: (x == "TERRORIST").sum())
ticks_df["ct_alive"] = ticks_df.groupby("tick")["team_name"].transform(lambda x: (x == "CT").sum())

# target = winner do round
ticks_df["target"] = ticks_df["round_win_status"]

In [116]:
ticks_df.head()

,is_bomb_planted,round_win_status,health,team_name,X,Y,weapon_name,tick,steamid,name,seconds_elapsed_in_round,round_win_status_label,round_win_reason_label,t_alive,ct_alive,target
11800,False,0,100,TERRORIST,-1850.0,1256.000000,Butterfly Knife,1180,76561197975129851,Sonic,0.0,in_progress,none,5,5,0
11801,False,0,100,CT,1600.0,-1440.000000,Karambit,1180,76561198365118288,try,0.0,in_progress,none,5,5,0
11802,False,0,100,CT,1456.0,-1328.000000,M9 Bayonet,1180,76561198067763828,chelo,0.0,in_progress,none,5,5,0
11803,False,0,100,TERRORIST,-2000.0,1434.233154,Karambit,1180,76561198060483793,br0,0.0,in_progress,none,5,5,0
11804,False,0,100,CT,1462.0,-1226.000000,Classic Knife,1180,76561197996370184,VINI,0.0,in_progress,none,5,5,0


# Train

In [ ]:
X = ticks_df[["t_alive", "ct_alive", "seconds_elapsed_in_round", "is_bomb_planted"]]
y = ticks_df["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model = GradientBoostingClassifier(verbose=1)
model.fit(X_train, y_train)

TypeError: BaseGradientBoosting.fit() got an unexpected keyword argument 'verbose'